<a href="https://colab.research.google.com/github/abiaggi/artificial-flavor-gpt2/blob/main/Artificial_Flavor_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys
import requests
from tqdm import tqdm

path = "json"
file_name = "cards.json"

if not os.path.exists(path):
    os.makedirs(path)
    
#url = "https://mtgjson.com/api/v5/AtomicCards.json"
url = "https://mtgjson.com/api/v5/AllPrintings.json"
# url = "https://mtgjson.com/api/v5/STA.json"
r = requests.get(url, stream=True)
with open(os.path.join(path, file_name), 'wb') as f:
    f.write(r.content)

In [2]:
import json
with open(os.path.join(path, file_name), encoding='utf-8') as f:
    all_cards = json.load(f)

flavorTexts = {}

for set_exp in all_cards['data']:
    set_cards = all_cards['data'][set_exp]['cards'];
    for card in set_cards:
        if('flavorText' in card):
            flavorTexts[card['name']] = card['flavorText']

In [3]:
if not os.path.exists('out'):
    os.makedirs('out')
with open(os.path.join('out', 'flavor.txt'), 'w', encoding='utf-8') as f:
    for card_name in flavorTexts:
        f.write(' '.join(flavorTexts[card_name].split()) + '\n')

In [4]:
model = "117M"

subdir = os.path.join('models', model)
if not os.path.exists(subdir):
    os.makedirs(subdir)
subdir = subdir.replace('\\','/') # needed for Windows

for filename in ['checkpoint','encoder.json','hparams.json','model.ckpt.data-00000-of-00001', 'model.ckpt.index', 'model.ckpt.meta', 'vocab.bpe']:

    r = requests.get("https://openaipublic.blob.core.windows.net/gpt-2/" + subdir + "/" + filename, stream=True)

    with open(os.path.join(subdir, filename), 'wb') as f:
        file_size = int(r.headers["content-length"])
        chunk_size = 1000
        with tqdm(ncols=100, desc="Fetching " + filename, total=file_size, unit_scale=True) as pbar:
            # 1k for chunk_size, since Ethernet packet size is around 1500 bytes
            for chunk in r.iter_content(chunk_size=chunk_size):
                f.write(chunk)
                pbar.update(chunk_size)

Fetching checkpoint: 1.00kit [00:00, 349kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 3.18Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 592kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:17, 29.2Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 3.19Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 2.10Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 2.05Mit/s]                                                       


GPT 2 Requires Tensorflow 1.*

In [7]:
pip install tensorflow==1.15

     |████████████████████████████████| 412.3MB 27kB/s 
     |████████████████████████████████| 3.8MB 46.6MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
     |████████████████████████████████| 512kB 53.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=3696fd08a3f8225f12e262b2359e61a3fd47501ecb7035de0567c3cbac45f272
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tens

In [8]:
pip install gpt2-client

  Created wheel for gpt2-client: filename=gpt2_client-2.1.5-cp37-none-any.whl size=16434 sha256=88336d0d3b418d9582087be9ea9e4559bcaab3766373d29d0146b89421dcca05
  Stored in directory: /root/.cache/pip/wheels/47/03/99/deadcb428f1b3c591edb8d6a2ac0136bb4326a266364b72153
  Created wheel for gpt-2-simple: filename=gpt_2_simple-0.7.2-cp37-none-any.whl size=23621 sha256=e20a54c566d6fed90807d132791d6e855d7f59269c0f0b0bc7b2acd55a0f7c8f
  Stored in directory: /root/.cache/pip/wheels/68/1d/15/c87a4302a6c7273ce1b4f282bec3c6877fb2f521535d87d30f
Successfully built gpt2-client gpt-2-simple


In [9]:
from gpt2_client import GPT2Client

gpt2 = GPT2Client('117M')  # This could also be `345M`, `774M`, or `1558M`.
gpt2.load_model(force_download=False)  # Use cached versions if available.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Loading checkpoint                             File already exists 
Loading encoder.json                           File already exists 
Loading hparams.json                           File already exists 
Loading model.ckpt.data-00000-of-00001         File already exists 
Loading model.ckpt.index                       File already exists 
Loading model.ckpt.meta                        File already exists 
Loading vocab.bpe                              File already exists 


In [10]:
corpus = "out/flavor.txt"
custom = gpt2.finetune(corpus, return_text=True)

Loading checkpoint models/117M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:02<00:00,  2.01s/it]


dataset has 319753 tokens
Training...
[1 | 9.22] loss=3.90 avg=3.90
[2 | 11.35] loss=3.77 avg=3.84
[3 | 13.49] loss=3.72 avg=3.80
[4 | 15.64] loss=3.55 avg=3.74
[5 | 17.79] loss=3.75 avg=3.74
[6 | 19.94] loss=3.90 avg=3.77
[7 | 22.10] loss=3.86 avg=3.78
[8 | 24.25] loss=3.81 avg=3.78
[9 | 26.41] loss=3.60 avg=3.76
[10 | 28.58] loss=3.70 avg=3.76
[11 | 30.75] loss=3.75 avg=3.76
[12 | 32.91] loss=3.76 avg=3.76
[13 | 35.08] loss=3.46 avg=3.73
[14 | 37.26] loss=3.55 avg=3.72
[15 | 39.44] loss=3.46 avg=3.70
[16 | 41.63] loss=3.63 avg=3.69
[17 | 43.81] loss=3.52 avg=3.68
[18 | 46.01] loss=3.47 avg=3.67
[19 | 48.21] loss=3.55 avg=3.66
[20 | 50.41] loss=3.64 avg=3.66
[21 | 52.62] loss=3.69 avg=3.66
[22 | 54.82] loss=3.44 avg=3.65
[23 | 57.04] loss=3.56 avg=3.65
[24 | 59.25] loss=3.46 avg=3.64
[25 | 61.47] loss=3.63 avg=3.64
[26 | 63.70] loss=3.54 avg=3.64
[27 | 65.92] loss=3.49 avg=3.63
[28 | 68.16] loss=3.69 avg=3.63
[29 | 70.40] loss=3.67 avg=3.63
[30 | 72.64] loss=3.64 avg=3.63
[31 | 74.89]